In [1]:
import numpy as np
from auxiliary.Model import Model

In [8]:
deep_param = {
    "beta" : 0.96,
    "gamma": 0.05,
    "rho" : 0.75, 
    "sigma" : 0.3
}

discretization_param = {
    "size_shock_grid" : 4, 
    "range_shock_grid" : 2.575
}

approx_param = {
    "max_iter" : 1000, 
    "precision" : 1e-7, 
    "size_capital_grid" : 11, 
}

In [3]:
model = Model(deep_param, discretization_param)


In [13]:
a = model._create_capital_grid(0.5,0.05, 5)

In [21]:
b, c = a[0][[0,2]]


In [22]:
b

20.29112160566703

In [25]:
a = (6*
    7)**5